In [2]:
import pandas as pd
import sklearn
import numpy as np

In [31]:
df = pd.read_csv("../Binance/btcusdt.csv")
df

,Unnamed: 0,Open,Close,High,Low
0,0,16530,16529,16530,16529
1,1,16529,16528,16529,16528
2,2,16528,16527,16528,16527
3,3,16527,16526,16527,16526
4,4,16526,16525,16526,16525
...,...,...,...,...,...
19297,19297,17349,17348,17349,17348
19298,19298,17348,17347,17348,17347
19299,19299,17347,17346,17347,17346
19300,19300,17346,17345,17346,17345


In [32]:
# #Check if any zero volumes are available
# indexZeros = df[ df['Volume'] == 0 ].index

# df.drop(indexZeros , inplace=True)
# df.loc[(df["Volume"] == 0 )]
# df.isna().sum()

In [33]:
df.head()

,Unnamed: 0,Open,Close,High,Low
0,0,16530,16529,16530,16529
1,1,16529,16528,16529,16528
2,2,16528,16527,16528,16527
3,3,16527,16526,16527,16526
4,4,16526,16525,16526,16525


In [34]:
import pandas_ta as ta

In [35]:
df['ATR'] = df.ta.atr(length=20)
df['RSI'] = df.ta.rsi()
df['EMA13'] = df.ta.ema(length=13)
df['EMA21'] = df.ta.ema(length=21)

from scipy.stats import linregress
def get_slope(array):
    y = np.array(array)
    x = np.arange(len(y))
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    return slope

In [36]:
backrollingN = 8

In [37]:
df['slopeRSI'] = df['RSI'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['slopeEMA13'] = df['EMA13'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['slopeEMA21'] = df['EMA21'].rolling(window=backrollingN).apply(get_slope, raw=True)

In [38]:
df = df.dropna()

In [39]:
df

,Unnamed: 0,Open,Close,High,Low,ATR,RSI,EMA13,EMA21,slopeRSI,slopeEMA13,slopeEMA21
27,27,16503,16502,16503,16502,1.000000,0.000000,16508.000000,16512.000000,0.000000,-1.000000,-1.000000
28,28,16502,16501,16502,16501,1.000000,0.000000,16507.000000,16511.000000,0.000000,-1.000000,-1.000000
29,29,16501,16500,16501,16500,1.000000,0.000000,16506.000000,16510.000000,0.000000,-1.000000,-1.000000
30,30,16500,16499,16500,16499,1.000000,0.000000,16505.000000,16509.000000,0.000000,-1.000000,-1.000000
31,31,16499,16498,16499,16498,1.000000,0.000000,16504.000000,16508.000000,0.000000,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
19297,19297,17349,17348,17349,17348,1.204677,47.507878,17349.029809,17348.972789,1.157401,0.017278,0.016252
19298,19298,17348,17347,17348,17347,1.194443,44.694143,17348.739836,17348.793445,0.221248,0.028415,0.023432
19299,19299,17347,17346,17347,17346,1.184721,42.014358,17348.348431,17348.539495,-1.240367,-0.030066,-0.013330
19300,19300,17346,17345,17346,17345,1.175485,39.466018,17347.870084,17348.217723,-2.196044,-0.129512,-0.078135


In [40]:
#Target flexible way
pipdiff = 500*1e-5 #for TP
SLTPRatio = 2 #pipdiff/Ratio gives SL

def mytarget(barsupfront, df1):
    length = len(df1)
    high = list(df1['High'])
    low = list(df1['Low'])
    close = list(df1['Close'])
    open = list(df1['Open'])
    trendcat = [None] * length
    
    for line in range (0,length-barsupfront-2):
        valueOpenLow = 0
        valueOpenHigh = 0
        for i in range(1,barsupfront+2):
            value1 = open[line+1]-low[line+i]
            value2 = open[line+1]-high[line+i]
            valueOpenLow = max(value1, valueOpenLow)
            valueOpenHigh = min(value2, valueOpenHigh)

            if ( (valueOpenLow >= pipdiff) and (-valueOpenHigh <= (pipdiff/SLTPRatio)) ):
                trendcat[line] = 1 #-1 downtrend
                break
            elif ( (valueOpenLow <= (pipdiff/SLTPRatio)) and (-valueOpenHigh >= pipdiff) ):
                trendcat[line] = 2 # uptrend
                break
            else:
                trendcat[line] = 0 # no clear trend
            
    return trendcat

In [41]:
# mytarget(barsfront to take into account, dataframe)
df['mytarget'] = mytarget(16, df)
df.tail()

C:\Users\hites\AppData\Local\Temp/ipykernel_31196/2438764458.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mytarget'] = mytarget(16, df)


,Unnamed: 0,Open,Close,High,Low,ATR,RSI,EMA13,EMA21,slopeRSI,slopeEMA13,slopeEMA21,mytarget
19297,19297,17349,17348,17349,17348,1.204677,47.507878,17349.029809,17348.972789,1.157401,0.017278,0.016252,NaN
19298,19298,17348,17347,17348,17347,1.194443,44.694143,17348.739836,17348.793445,0.221248,0.028415,0.023432,NaN
19299,19299,17347,17346,17347,17346,1.184721,42.014358,17348.348431,17348.539495,-1.240367,-0.030066,-0.013330,NaN
19300,19300,17346,17345,17346,17345,1.175485,39.466018,17347.870084,17348.217723,-2.196044,-0.129512,-0.078135,NaN
19301,19301,17345,17344,17345,17344,1.166711,37.046173,17347.317215,17347.834293,-2.815774,-0.250466,-0.159777,NaN


In [42]:
df = df.dropna()
attributes = ['ATR', 'RSI', 'EMA13', 'slopeRSI', 'slopeEMA13']
x = df[attributes]
y = df['mytarget']
print(x)

            ATR        RSI         EMA13  slopeRSI  slopeEMA13
27     1.000000   0.000000  16508.000000  0.000000   -1.000000
28     1.000000   0.000000  16507.000000  0.000000   -1.000000
29     1.000000   0.000000  16506.000000  0.000000   -1.000000
30     1.000000   0.000000  16505.000000  0.000000   -1.000000
31     1.000000   0.000000  16504.000000  0.000000   -1.000000
...         ...        ...           ...       ...         ...
19279  1.109910  62.725932  17348.047933  3.420922    0.164228
19280  1.104415  65.153207  17348.612514  2.907820    0.283624
19281  1.149194  57.139018  17348.810726  1.886300    0.350249
19282  1.141734  53.589586  17348.837765  0.641785    0.358037
19283  1.184647  59.066319  17349.146656  0.073030    0.344304

[19257 rows x 5 columns]


In [43]:
df.isna().sum()

Unnamed: 0    0
Open          0
Close         0
High          0
Low           0
ATR           0
RSI           0
EMA13         0
EMA21         0
slopeRSI      0
slopeEMA13    0
slopeEMA21    0
mytarget      0
dtype: int64

In [44]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=2)

In [45]:
from sklearn.svm import SVC

In [46]:
model = SVC()

In [47]:
model.fit(xtrain, ytrain)

y_pred_train = model.predict(xtrain)
y_pred_test = model.predict(xtest)

In [48]:
from sklearn.metrics import accuracy_score
accurancy_train = accuracy_score(ytrain, y_pred_train)
accuracy_tesst = accuracy_score(ytest, y_pred_test)

print("accuracy train %.f%% "% (accurancy_train*100))
print("accuracy test %.f%% "% (accuracy_tesst*100))

accuracy train 52% 
accuracy test 51% 


In [49]:
print(df['mytarget'].value_counts()*100/df['mytarget'].count())


pred_test = np.random.choice([0,1,2], len(y_pred_test))
# accuracy_tesst = accuracy_score(ytest, pred_test)
# print("accuracy gambler %.2f%%" %(accuracy_tesst*100))

2.0    52.204393
1.0    47.795607
Name: mytarget, dtype: float64


In [50]:
train_index = int(0.8 * len(x))
xtrain, xtest = x[:train_index], x[train_index:]
ytrain, ytest = y[:train_index], y[train_index:]

In [51]:
model2 = SVC()
model2.fit(xtrain, ytrain)

y_pred_train = model.predict(xtrain)
y_pred_test = model.predict(xtest)

In [52]:
from sklearn.metrics import accuracy_score
accurancy_train = accuracy_score(ytrain, y_pred_train)
accuracy_tesst = accuracy_score(ytest, y_pred_test)

print("accuracy train %.f%% "% (accurancy_train*100))
print("accuracy test %.f%% "% (accuracy_tesst*100))

accuracy train 51% 
accuracy test 55% 


In [53]:
print(df['mytarget'].value_counts()*100/df['mytarget'].count())


pred_test = np.random.choice([0,1,2], len(y_pred_test))
accuracy_tesst = accuracy_score(ytest, pred_test)
print("accuracy gambler %.2f%%" %(accuracy_tesst*100))

2.0    52.204393
1.0    47.795607
Name: mytarget, dtype: float64
accuracy gambler 32.48%


In [57]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd



# Create the contingency mat
matrix_test = confusion_matrix(ytest, y_pred_test)
matrix_train = confusion_matrix(ytrain, y_pred_train)
classification_report_pred = classification_report(ytest, y_pred_test, target_names=["downtrend", "uptrend"])

# Convert the matrix to a DataFrame for easy display

matrix_df_test = pd.DataFrame(matrix_test, columns=['pred_Downtrend','pred_Uptrend'], index=['Downtrend','Uptrend'])
# matrix_df_test = pd.DataFrame(matrix_test, columns=['pred_sideways','pred_Downtrend','pred_Uptrend'], index=['Sidetrend','Downtrend','Uptrend'])

matrix_df_train = pd.DataFrame(matrix_train, columns=['pred_Downtrend','pred_Uptrend'], index=['Downtrend','Uptrend'])
# matrix_df_train = pd.DataFrame(matrix_train, columns=['pred_sideways','pred_Downtrend','pred_Uptrend'], index=['Sidetrend','Downtrend','Uptrend'])
print("Test-Set Confusion Matrix\n")
print(matrix_df_test)
print("-----------------------------------------------------------------------------")
print("Train-Set Confusion Matrix\n")
print(matrix_df_train)
print("-----------------------------------------------------------------------------")
print("Classification Report\n")
print(classification_report_pred)

Test-Set Confusion Matrix

           pred_Downtrend  pred_Uptrend
Downtrend               0          1715
Uptrend                 0          2137
-----------------------------------------------------------------------------
Train-Set Confusion Matrix

           pred_Downtrend  pred_Uptrend
Downtrend               0          7489
Uptrend                 0          7916
-----------------------------------------------------------------------------
Classification Report

              precision    recall  f1-score   support

   downtrend       0.00      0.00      0.00      1715
     uptrend       0.55      1.00      0.71      2137

    accuracy                           0.55      3852
   macro avg       0.28      0.50      0.36      3852
weighted avg       0.31      0.55      0.40      3852



C:\Users\hites\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hites\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hites\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
# Import necessary libraries
import pandas as pd

# Define brick size
brick_size = 0.5

# Read in candlestick data
df = pd.read_csv("./BTCUSD_Candlestick_1_M_BID_01.01.2023-09.01.2023.csv")


In [57]:
df = df.head(1000)

In [58]:
df

,Local time,Open,High,Low,Close,Volume
0,01.01.2023 00:00:00.000 GMT+0530,16530,16530,16530,16530,0.0
1,01.01.2023 00:01:00.000 GMT+0530,16530,16530,16530,16530,0.0
2,01.01.2023 00:02:00.000 GMT+0530,16530,16530,16530,16530,0.0
3,01.01.2023 00:03:00.000 GMT+0530,16530,16530,16530,16530,0.0
4,01.01.2023 00:04:00.000 GMT+0530,16530,16530,16530,16530,0.0
...,...,...,...,...,...,...
995,01.01.2023 16:35:00.000 GMT+0530,16530,16530,16530,16530,0.0
996,01.01.2023 16:36:00.000 GMT+0530,16530,16530,16530,16530,0.0
997,01.01.2023 16:37:00.000 GMT+0530,16530,16530,16530,16530,0.0
998,01.01.2023 16:38:00.000 GMT+0530,16530,16530,16530,16530,0.0


In [61]:
# Create Renko chart
renko_df = pd.DataFrame(columns=['Local time', 'Open', 'High', 'Low', 'Close'])
prev_close = df.iloc[0]['Close']
brick_val = prev_close
for index, row in df.iterrows():
    if abs(row['Close'] - prev_close) >= brick_size:
        direction = 1 if row['Close'] > prev_close else -1
        while abs(row['Close'] - brick_val) >= brick_size:
            renko_df = renko_df.append({'Local time': row['Local time'], 'Open': brick_val, 'High': brick_val + direction * brick_size, 'Low': brick_val, 'Close': brick_val + direction * brick_size}, ignore_index=True)
            brick_val += direction * brick_size
        prev_close = row['Close']

In [62]:
renko_df

,Local time,Open,High,Low,Close
